In [1]:
import os
from pathlib import Path
import pandas as pd
import re
import requests

## Logic
1. Take a union of all the three dfs
2. Identify unique ensembl IDs, and check all gene names associated with each id. Check if it's a one to one mapping
3. Do the vice versa too, as a sanity check

In [2]:
# Set filepaths

filepath1 = Path('/mnt/share/cfdeworkbench/C2M2/ontology/external_CV_reference_files_updated_202412')
filepath2 = Path('/mnt/share/cfdeworkbench/C2M2/ontology/external-CV-reference-scripts/Ensembl_preprocessing/002_all')
filepath3 = Path('/mnt/share/cfdeworkbench/C2M2/ontology')

# Read all files as dataframe
df1 = pd.read_csv(filepath1/'ensembl_genes.tsv', delimiter='\t')
df2 = pd.read_csv(filepath2/'ensembl_genes.tsv', delimiter='\t')
df3 = pd.read_csv(filepath3/'C2M2_genes.tsv', delimiter='\t')

In [3]:
df1

,id,name,description,synonyms,organism
0,ENSG00000000003,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],"[""ENTREZ:7105"", ""HGNC:11858"", ""MIM:300191"", ""N...",NCBI:txid9606
1,ENSG00000000005,TNMD,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],"[""BRICD4"", ""CHM1L"", ""ENTREZ:64102"", ""HGNC:1775...",NCBI:txid9606
2,ENSG00000000419,DPM1,dolichyl-phosphate mannosyltransferase subunit...,"[""CDGIE"", ""DPM1"", ""ENTREZ:8813"", ""HGNC:3005"", ...",NCBI:txid9606
3,ENSG00000000457,SCYL3,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,"[""ENTREZ:57147"", ""HGNC:19285"", ""MIM:608192"", ""...",NCBI:txid9606
4,ENSG00000000460,C1orf112,chromosome 1 open reading frame 112 [Source:HG...,"[""C1orf112"", ""ENTREZ:55732"", ""HGNC:25565"", ""NM...",NCBI:txid9606
...,...,...,...,...,...
24637,ENSG00000291313,NaN,novel protein,[],NCBI:txid9606
24638,ENSG00000291314,NaN,novel protein,[],NCBI:txid9606
24639,ENSG00000291315,NaN,novel protein,[],NCBI:txid9606
24640,ENSG00000291316,NaN,"novel protein, LOC84773-CYHR1 readthrough",[],NCBI:txid9606


In [4]:
df2

,id,name,description,synonyms,organism
0,ENSG00000000003,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],"[""ENTREZ:7105"", ""HGNC:11858"", ""MIM:300191"", ""N...",NCBI:txid9606
1,ENSG00000000005,TNMD,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],"[""BRICD4"", ""CHM1L"", ""ENTREZ:64102"", ""HGNC:1775...",NCBI:txid9606
2,ENSG00000000419,DPM1,dolichyl-phosphate mannosyltransferase subunit...,"[""CDGIE"", ""DPM1"", ""ENTREZ:8813"", ""HGNC:3005"", ...",NCBI:txid9606
3,ENSG00000000457,SCYL3,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,"[""ENTREZ:57147"", ""HGNC:19285"", ""MIM:608192"", ""...",NCBI:txid9606
4,ENSG00000000460,FIRRM,FIGNL1 interacting regulator of recombination ...,"[""C1orf112"", ""ENTREZ:55732"", ""HGNC:25565"", ""NM...",NCBI:txid9606
...,...,...,...,...,...
21561,ENSG00000293689,NaN,novel protein,[],NCBI:txid9606
21562,ENSG00000293691,NaN,novel protein,[],NCBI:txid9606
21563,ENSG00000300510,NaN,novel protein similar to interferon induced tr...,[],NCBI:txid9606
21564,ENSG00000310517,CAST,calpastatin [Source:NCBI gene (formerly Entrez...,[],NCBI:txid9606


In [22]:
df2.loc[df2['id'] == 'ENSG00000310517']

,id,name,description,synonyms,organism
21564,ENSG00000310517,CAST,calpastatin [Source:NCBI gene (formerly Entrez...,[],NCBI:txid9606


In [6]:
union_df = pd.concat([df1,df2,df3], axis=0, ignore_index=True)
union_df

,id,name,description,synonyms,organism
0,ENSG00000000003,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],"[""ENTREZ:7105"", ""HGNC:11858"", ""MIM:300191"", ""N...",NCBI:txid9606
1,ENSG00000000005,TNMD,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],"[""BRICD4"", ""CHM1L"", ""ENTREZ:64102"", ""HGNC:1775...",NCBI:txid9606
2,ENSG00000000419,DPM1,dolichyl-phosphate mannosyltransferase subunit...,"[""CDGIE"", ""DPM1"", ""ENTREZ:8813"", ""HGNC:3005"", ...",NCBI:txid9606
3,ENSG00000000457,SCYL3,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,"[""ENTREZ:57147"", ""HGNC:19285"", ""MIM:608192"", ""...",NCBI:txid9606
4,ENSG00000000460,C1orf112,chromosome 1 open reading frame 112 [Source:HG...,"[""C1orf112"", ""ENTREZ:55732"", ""HGNC:25565"", ""NM...",NCBI:txid9606
...,...,...,...,...,...
66529,ENSG00000285913,FAM90A18P,"family with sequence similarity 90 member A18,...",[],NCBI:txid9606
66530,ENSG00000285937,FAM90A8P,"family with sequence similarity 90 member A8, ...",[],NCBI:txid9606
66531,ENSG00000285950,FAM90A10P,"family with sequence similarity 90 member A10,...",[],NCBI:txid9606
66532,ENSG00000286237,ARMCX5-GPRASP2,ARMCX5-GPRASP2 readthrough [Source:NCBI gene (...,[],NCBI:txid9606


In [7]:
# Define a regex pattern for extracting synonyms
pattern = r'"(.*?)"'

# Parse synonyms into lists and count their lengths
union_df['parsed_synonyms'] = union_df['synonyms'].apply(
    lambda x: re.findall(pattern, x) if isinstance(x, str) else []
)
union_df['synonym_count'] = union_df['parsed_synonyms'].apply(len)

# Sort by descending synonym count
union_df = union_df.sort_values(by='synonym_count', ascending=False)

# Step 2: Drop rows with lesser synonyms for same id and name
# Group by 'id' and 'name', keep the row with max synonym count
filtered_df = (
    union_df.sort_values(by='synonym_count', ascending=False)
    .drop_duplicates(subset=['id', 'name'], keep='first')
)

# Step 3: Find conflicting rows where one id has multiple names
# Group by 'id' and check unique names
conflicting_ids = (
    union_df.groupby('id')['name']
    .nunique()
    .reset_index(name='unique_name_count')
    .query('unique_name_count > 1')['id']
)

# Extract rows with conflicting IDs
conflicting_rows_df = union_df[union_df['id'].isin(conflicting_ids)]

# Output: filtered_df and conflicting_rows_df
print("Filtered DataFrame with duplicates dropped:")
print(filtered_df)

print("\nConflicting rows where one ID has multiple names:")
print(conflicting_rows_df)

Filtered DataFrame with duplicates dropped:
                    id             name  \
46533  ENSG00000047849             MAP4   
51684  ENSG00000169398             PTK2   
54210  ENSG00000110514             MADD   
9436   ENSG00000152601            MBNL1   
46926  ENSG00000076864          RAP1GAP   
...                ...              ...   
18479  ENSG00000225528              NaN   
56714  ENSG00000289720  ENSG00000289720   
56710  ENSG00000288674  ENSG00000288674   
20596  ENSG00000269167              NaN   
66135  ENSG00000263464          PPIAL4C   

                                             description  \
46533  microtubule associated protein 4 [Source:HGNC ...   
51684  protein tyrosine kinase 2 [Source:HGNC Symbol;...   
54210  MAP kinase activating death domain [Source:HGN...   
9436   muscleblind like splicing regulator 1 [Source:...   
46926  RAP1 GTPase activating protein [Source:HGNC Sy...   
...                                                  ...   
18479  novel protei

In [8]:
# Drop duplicates based on 'id'
no_conflict_df = conflicting_rows_df.drop_duplicates(subset='id', keep='first')

# Function to fetch the official name from the Ensembl API
def fetch_and_update_name(row):
    ensembl_id = row['id']
    url = f"https://rest.ensembl.org/lookup/id/{ensembl_id}?content-type=application/json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        official_name = data.get("display_name", None)  # Fetch official name
        if official_name and row['name'] != official_name:
            return official_name  # Update with the official name if it differs
        return row['name']  # Keep the original name if it matches
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ensembl_id}: {e}")
        return row['name']  # Keep the original name if API call fails

# Apply the function to update names if they don't match the official names
no_conflict_df['name'] = no_conflict_df.apply(fetch_and_update_name, axis=1)

# Output the updated DataFrame
print(no_conflict_df)

                    id      name  \
58295  ENSG00000113391     ARB2A   
56662  ENSG00000284762     PDE8B   
29243  ENSG00000116922     AIRIM   
28703  ENSG00000112739     PRP4K   
614    ENSG00000047346     ATOSA   
...                ...       ...   
39609  ENSG00000183889    NPIPA6   
6636   ENSG00000133136     GNG5B   
18811  ENSG00000230045  FAM90A15   
19038  ENSG00000233295  FAM90A20   
19025  ENSG00000233132   FAM90A3   

                                             description  \
58295  family with sequence similarity 172 member A [...   
56662  phosphodiesterase 8B [Source:NCBI gene (former...   
29243  AFG2 interacting ribosome maturation factor [S...   
28703  pre-mRNA processing factor kinase PRP4K [Sourc...   
614    atos homolog A [Source:HGNC Symbol;Acc:HGNC:25...   
...                                                  ...   
39609  nuclear pore complex interacting protein famil...   
6636   G protein subunit gamma 5B [Source:HGNC Symbol...   
18811  family with sequence

/tmp/ipykernel_3124298/3662977809.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_conflict_df['name'] = no_conflict_df.apply(fetch_and_update_name, axis=1)


In [9]:
# Concatenate the filtered and conflicting DataFrames
final_df = pd.concat([filtered_df, no_conflict_df], axis=0, ignore_index=True)

# Drop the 'parsed_synonyms' and 'synonym_count' columns
final_df = final_df.drop(columns=['parsed_synonyms', 'synonym_count'], errors='ignore')

# Sort by 'id' (assuming 'id' is the Ensembl ID column)
final_df = final_df.sort_values(by='id', ascending=True)

# Reset the index
final_df = final_df.reset_index(drop=True)

# Output the final DataFrame
print(final_df)

                    id    name  \
0      ENSG00000000003  TSPAN6   
1      ENSG00000000005    TNMD   
2      ENSG00000000419    DPM1   
3      ENSG00000000457   SCYL3   
4      ENSG00000000460   FIRRM   
...                ...     ...   
25233  ENSG00000293689     NaN   
25234  ENSG00000293691     NaN   
25235  ENSG00000300510     NaN   
25236  ENSG00000310517    CAST   
25237  ENSG00000310525     NaN   

                                             description  \
0      tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]   
1        tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]   
2      dolichyl-phosphate mannosyltransferase subunit...   
3      SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...   
4      FIGNL1 interacting regulator of recombination ...   
...                                                  ...   
25233                                      novel protein   
25234                                      novel protein   
25235  novel protein similar to interferon induced 

In [ ]:
final_df.to_csv('')

,id,name,description,synonyms,organism
0,ENSG00000000003,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],"[""ENTREZ:7105"", ""HGNC:11858"", ""MIM:300191"", ""N...",NCBI:txid9606
1,ENSG00000000005,TNMD,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],"[""BRICD4"", ""CHM1L"", ""ENTREZ:64102"", ""HGNC:1775...",NCBI:txid9606
2,ENSG00000000419,DPM1,dolichyl-phosphate mannosyltransferase subunit...,"[""CDGIE"", ""DPM1"", ""ENTREZ:8813"", ""HGNC:3005"", ...",NCBI:txid9606
3,ENSG00000000457,SCYL3,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,"[""ENTREZ:57147"", ""HGNC:19285"", ""MIM:608192"", ""...",NCBI:txid9606
4,ENSG00000000460,FIRRM,FIGNL1 interacting regulator of recombination ...,"[""C1orf112"", ""ENTREZ:55732"", ""HGNC:25565"", ""NM...",NCBI:txid9606
...,...,...,...,...,...
25233,ENSG00000293689,NaN,novel protein,[],NCBI:txid9606
25234,ENSG00000293691,NaN,novel protein,[],NCBI:txid9606
25235,ENSG00000300510,NaN,novel protein similar to interferon induced tr...,[],NCBI:txid9606
25236,ENSG00000310517,CAST,calpastatin [Source:NCBI gene (formerly Entrez...,[],NCBI:txid9606


In [3]:
import requests, sys
 
server = "https://rest.ensembl.org"
ext = "/xrefs/id/ENSG00000000003"
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
synonyms = decoded[3]['synonyms']
print(synonyms)

['T245', 'TM4SF6', 'TSPAN-6']


In [17]:
import pandas as pd
import json

# JSON data
data = [
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "T245",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TM4SF6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN-6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278740",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278741",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278742",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_001278743",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NM_003270",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265669",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265670",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265671",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_001265672",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "NP_003261",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XM_011531018",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A024RCI0",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WZU5",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "A0A087WYV6",
    "KEGG": "7105",
    "MARRVEL": "7105"
  },
  {
    "ENSEMBL.VER": "ENSG00000000003",
    "ENSEMBL": "ENSG00000000003",
    "SYMBOL": "TSPAN6",
    "ENTREZID": "7105",
    "ALIAS": "TSPAN6",
    "GENENAME": "tetraspanin 6",
    "REFSEQ": "XP_011529320",
    "UNIPROT": "O43657",
    "KEGG": "7105",
    "MARRVEL": "7105"
  }
]

# Load JSON data into a DataFrame
df = pd.DataFrame(data)

# Extract the required fields
columns_to_extract = ["GENENAME", "REFSEQ", "UNIPROT", "KEGG"]

# Group by ENSEMBL, deduplicate, and concatenate synonyms
df_synonyms = (
    df.groupby("ENSEMBL")[columns_to_extract]
    .agg(lambda x: sorted(set(filter(None, x))))
    .reset_index()
)

# Convert the combined fields into a single list of strings with quotes and brackets
def format_synonyms(row):
    all_synonyms = [f'"{syn}"' for col in row for syn in col if syn]
    return f"[{', '.join(all_synonyms)}]"

df_synonyms["SYNONYMS"] = df_synonyms[columns_to_extract].apply(format_synonyms, axis=1)

# Select only ENSEMBL and SYNONYMS columns for output
df_result = df_synonyms[["ENSEMBL", "SYNONYMS"]]

# Save to a file
output_file = "synonyms_resolved.tsv"
df_result.to_csv(output_file, sep="\t", index=False)



In [19]:
import pandas as pd
import requests

# Define a function to extract synonyms from the JSON response
def extract_synonyms(data):
    """
    Extract synonyms from the JSON response data.
    Combine GENENAME, REFSEQ, UNIPROT, KEGG fields into a formatted list.
    """
    columns_to_extract = ["GENENAME", "REFSEQ", "UNIPROT", "KEGG"]

    # Convert JSON data into a DataFrame
    df = pd.DataFrame(data)

    # Group by ENSEMBL, deduplicate, and format synonyms
    def format_synonyms(row):
        all_synonyms = [f'"{syn}"' for col in row for syn in col if syn]
        return f"[{', '.join(all_synonyms)}]"

    df_synonyms = (
        df.groupby("ENSEMBL")[columns_to_extract]
        .agg(lambda x: sorted(set(filter(None, x))))
        .reset_index()
    )
    df_synonyms["SYNONYMS"] = df_synonyms[columns_to_extract].apply(format_synonyms, axis=1)

    # Return ENSEMBL and formatted synonyms
    return df_synonyms[["ENSEMBL", "SYNONYMS"]]

# Read the input file with the list of Ensembl IDs
input_file = "no_synonym_ids.txt"
ensembl_ids = pd.read_csv(input_file, sep="\t")["id"]

# Initialize a list to store results
results = []

# Iterate through each Ensembl ID and fetch data
base_url = "https://bdcw.org/geneid/rest/species/hsa/GeneIDType/ENSEMBL/GeneListStr/{}/USE_NCBI_GENE_INFO/0/View/json"
for ensembl_id in ensembl_ids:
    try:
        # Fetch data from the API
        url = base_url.format(ensembl_id)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the JSON response
        json_data = response.json()

        # Extract synonyms
        synonyms_df = extract_synonyms(json_data)
        results.append(synonyms_df)
    except Exception as e:
        print(f"Failed to process {ensembl_id}: {e}")

# Combine all results into a single DataFrame
if results:
    final_df = pd.concat(results, ignore_index=True)

    # Save the results to a TSV file
    output_file = "synonyms_resolved.tsv"
    final_df.to_csv(output_file, sep="\t", index=False)
    print(f"Synonyms saved to {output_file}")
else:
    print("No results were processed successfully.")


Synonyms saved to synonyms_resolved.tsv
